In [ ]:
import h5py
import numpy as np
import random
import pdb

f = h5py.File('data/Full_NF.h5', 'r')
dset = f['nf_data']
dset = dset[:]
print("Data loaded from file")

dset_indicies = set(range(len(dset)))
train_num = int(.9 * len(dset))
print("Training Samples: ", train_num, "| Validation Samples: ", len(dset) - train_num)

train_indicies = random.sample(dset_indicies, train_num)
validation_indicies = dset_indicies.difference(train_indicies)
training_data = [dset[i] for i in train_indicies]
validation_data = [dset[i] for i in validation_indicies]
print("Training and Validation datasets separated")

del dset_indicies
del train_indicies
del validation_indicies
del dset
        
def nf_label_transformation_function(label):
    
    one_hot = np.zeros(5)
    one_hot[label - 1] = 1
    return one_hot

import warnings
warnings.simplefilter('ignore')

import keras
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from generator_helper_functions import input_layer, naive_input_size , Generate_Naive_Feedforward_Contig, Generate_Naive_Feedforward

#opt = keras.optimizers.rmsprop()
opt = keras.optimizers.SGD()

batch_size = 64

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(naive_input_size,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])

print("Model Compiled")


In [ ]:
train_gen = Generate_Naive_Feedforward(training_data, 
                                batch_size, 
                                (naive_input_size,), 
                                (5,), #label shape
                                input_layer, 
                                lambda x: nf_label_transformation_function(x[2]),
                                normalized = True
                                )

validation_gen = Generate_Naive_Feedforward(validation_data, 
                                batch_size, 
                                (naive_input_size,), 
                                (5,), #label shape
                                input_layer, 
                                lambda x: nf_label_transformation_function(x[2]),
                                normalized = True
                                )

checkpointer = ModelCheckpoint(filepath='models/naive_feedforward_SGD_128_128v1_normalized.hdf5', verbose=1, save_best_only=True)
model.fit_generator(train_gen,
                    epochs=20,
                    steps_per_epoch=1000,
                    validation_data=validation_gen,
                    validation_steps=100,
                    callbacks=[checkpointer]
                    )

In [ ]:
import h5py
import numpy as np
import random
import pdb

f = h5py.File('data/Full_NF.h5', 'r')
dset = f['nf_data']
dset = dset[:]
print("Data loaded from file")

dset_indicies = set(range(len(dset)))
train_num = int(.9 * len(dset))
print("Training Samples: ", train_num, "| Validation Samples: ", len(dset) - train_num)

train_indicies = random.sample(dset_indicies, train_num)
validation_indicies = dset_indicies.difference(train_indicies)
training_data = [dset[i] for i in train_indicies]
validation_data = [dset[i] for i in validation_indicies]
print("Training and Validation datasets separated")

del dset_indicies
del train_indicies
del validation_indicies
del dset

def Generate_Naive_Feedforward(dataset, batch_size, feature_shape, feature_transformation_function, label_transformation_function):
    
    features = np.zeros((batch_size, *feature_shape))
    labels = np.zeros((batch_size, 5))
    index_set = set(range(len(dataset)))
    
    while True:
            
        batch_indicies = []    
        if (len(index_set) < batch_size):
            index_set = set(range(len(dataset)))
            batch_indicies = random.sample(index_set, batch_size)
        else:
            batch_indicies = random.sample(index_set, batch_size)
            index_set = index_set.difference(batch_indicies)
        
        for i, batch_index in enumerate(batch_indicies):
            features[i] = feature_transformation_function(dataset[batch_index])
            labels[i] = label_transformation_function(dataset[batch_index])
            
        yield (features, labels)
        
def nf_label_transformation_function(label):
    
    one_hot = np.zeros(5)
    one_hot[label - 1] = 1
    return one_hot

import warnings
warnings.simplefilter('ignore')

import keras
from keras.models import load_model
from generator_helper_functions import input_layer, naive_input_size

batch_size = 64

model = load_model('models/naive_feedforward_SGD_128_128v1.hdf5')

print("Loaded")

validation_gen = Generate_Naive_Feedforward(validation_data, 
                                batch_size, 
                                (naive_input_size,), 
                                input_layer, 
                                lambda x: nf_label_transformation_function(x[2]))

model.evaluate_generator(validation_gen,
                    steps=1000,
                    )

## Trying different neurons per hidden layer and regression

In [ ]:
import h5py
import numpy as np
import random
import pdb

f = h5py.File('data/Full_NF.h5', 'r')
dset = f['nf_data']
dset = dset[:]
print("Data loaded from file")

dset_indicies = set(range(len(dset)))
train_num = int(.9 * len(dset))
print("Training Samples: ", train_num, "| Validation Samples: ", len(dset) - train_num)

train_indicies = random.sample(dset_indicies, train_num)
validation_indicies = dset_indicies.difference(train_indicies)
training_data = [dset[i] for i in train_indicies]
validation_data = [dset[i] for i in validation_indicies]
print("Training and Validation datasets separated")

del dset_indicies
del train_indicies
del validation_indicies
del dset

def Generate_Naive_Feedforward(dataset, batch_size, feature_shape, feature_transformation_function, label_transformation_function, normalized = False):
    
    features = np.zeros((batch_size, *feature_shape))
    labels = np.zeros((batch_size,))
    index_set = set(range(len(dataset)))
    
    while True:
            
        batch_indicies = []    
        if (len(index_set) < batch_size):
            index_set = set(range(len(dataset)))
            batch_indicies = random.sample(index_set, batch_size)
        else:
            batch_indicies = random.sample(index_set, batch_size)
            index_set = index_set.difference(batch_indicies)
        
        for i, batch_index in enumerate(batch_indicies):
            features[i] = feature_transformation_function(dataset[batch_index], normalized)
            labels[i] = label_transformation_function(dataset[batch_index])
            
        yield (features, labels)

import warnings
warnings.simplefilter('ignore')

import keras
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense, Lambda
from keras.callbacks import ModelCheckpoint
from generator_helper_functions import input_layer, naive_input_size

#opt = keras.optimizers.rmsprop()
opt = keras.optimizers.SGD()

batch_size = 64

def round(inp):
    return tf.round(inp * 4) / 4

model = Sequential()
model.add(Dense(90, activation='relu', input_shape=(naive_input_size,)))
model.add(Dense(90, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.add(Lambda(round))

model.compile(loss='binary_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])

print("Model Compiled")

train_gen = Generate_Naive_Feedforward(training_data, 
                                batch_size, 
                                (naive_input_size,), 
                                input_layer, 
                                lambda x: (x[2] - 1)/4,
                                normalized=True)

validation_gen = Generate_Naive_Feedforward(validation_data, 
                                batch_size, 
                                (naive_input_size,), 
                                input_layer, 
                                lambda x: (x[2] - 1 )/4,
                                normalized=True)

checkpointer = ModelCheckpoint(filepath='models/naive_feedforward_regression_90_90_normalized.hdf5', verbose=1, save_best_only=True)
model.fit_generator(train_gen,
                    epochs=1,
                    steps_per_epoch=1000,
                    validation_data=validation_gen,
                    validation_steps=100,
                    callbacks=[checkpointer]
                    )

In [ ]:
model.predict(input_layer(training_data[0]))

In [ ]:
input_layer(training_data[0])

In [ ]:
model.predict()

## Trying different neurons per hidden layer and poisson regression

In [ ]:
import pickle

with open('../data/validation_data.pickle', 'rb') as handle:
    validation_data = pickle.load(handle)

with open('../data/training_data.pickle', 'rb') as handle:
    training_data = pickle.load(handle)

In [ ]:
len (validation_data)

In [ ]:
len(training_data)

In [1]:
import h5py
import numpy as np
import random
import pdb
import pickle

# f = h5py.File('../data/Full_NF.h5', 'r')
# dset = f['nf_data']
# dset = dset[:]
# print("Data loaded from file")

# dset_indicies = set(range(len(dset)))
# train_num = int(.9 * len(dset))
# print("Training Samples: ", train_num, "| Validation Samples: ", len(dset) - train_num)

# train_indicies = random.sample(dset_indicies, train_num)
# validation_indicies = dset_indicies.difference(train_indicies)
# training_data = [dset[i] for i in train_indicies]
# validation_data = [dset[i] for i in validation_indicies]
# print("Training and Validation datasets separated")

with open('../data/validation_data.pickle', 'rb') as handle:
    validation_data = pickle.load(handle)

with open('../data/training_data.pickle', 'rb') as handle:
    training_data = pickle.load(handle)

print("Training and Validation datasets separated")


# del dset_indicies
# del train_indicies
# del validation_indicies
# del dset

# def Generate_Naive_Feedforward(dataset, batch_size, feature_shape, feature_transformation_function, label_transformation_function):
    
#     features = np.zeros((batch_size, *feature_shape))
#     labels = np.zeros((batch_size,))
#     index_set = set(range(len(dataset)))
    
#     while True:
            
#         batch_indicies = []    
#         if (len(index_set) < batch_size):
#             index_set = set(range(len(dataset)))
#             batch_indicies = random.sample(index_set, batch_size)
#         else:
#             batch_indicies = random.sample(index_set, batch_size)
#             index_set = index_set.difference(batch_indicies)
        
#         for i, batch_index in enumerate(batch_indicies):
#             features[i] = feature_transformation_function(dataset[batch_index])
#             labels[i] = label_transformation_function(dataset[batch_index])
            
#         yield (features, labels)

import warnings
warnings.simplefilter('ignore')

import keras
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from generator_helper_functions import input_layer, naive_input_size, Generate_Naive_Feedforward

#opt = keras.optimizers.rmsprop()
opt = keras.optimizers.SGD()

batch_size = 128

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(int(naive_input_size),)))
model.add(Dense(64, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='relu'))

model.compile(loss='mean_squared_error',
             optimizer=opt,
             metrics=['accuracy'])

print("Model Compiled")

train_gen = Generate_Naive_Feedforward(training_data, 
                                batch_size, 
                                (naive_input_size,), 
                                (1,),
                                input_layer, 
                                lambda x: (x[2] - 1)/4,
                                normalized=True)

validation_gen = Generate_Naive_Feedforward(validation_data, 
                                batch_size, 
                                (naive_input_size,),
                                (1,),
                                input_layer, 
                                lambda x: (x[2] - 1 )/4,
                                normalized=True)

checkpointer = ModelCheckpoint(filepath='../models/naive_feedforward_regression_poisson_Native_64_932.hdf5', verbose=1, save_best_only=True)
model.fit_generator(train_gen,
                    epochs=1,
                    steps_per_epoch=1000,
                    validation_data=validation_gen,
                    validation_steps=10,
                    callbacks=[checkpointer]
                    )

/home/steich/NF_Prize_Data/nf_prize_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Training and Validation datasets separated


Using TensorFlow backend.


Model Compiled
Epoch 1/1
1000/1000 [==============================] - 2030s 2s/step - loss: 0.0750 - acc: 0.2238 - val_loss: 0.0632 - val_acc: 0.1695

Epoch 00001: val_loss improved from inf to 0.06321, saving model to ../models/naive_feedforward_regression_poisson_Native_64_932.hdf5


In [2]:
from sklearn.metrics import precision_score


In [10]:
precision_score(
    list(map(lambda x: round(float(model.predict(np.expand_dims(input_layer(x,   True), axis=0)) * 4 + 1)), 
             validation_data[0:3000])), 
    validation_data[0:3000,2], average='micro') 



0.33366666666666667

In [ ]:
import h5py
import numpy as np
import random
import pdb

f = h5py.File('data/Full_NF.h5', 'r')
dset = f['nf_data']
dset = dset[:]
print("Data loaded from file")

dset_indicies = set(range(len(dset)))
train_num = int(.9 * len(dset))
print("Training Samples: ", train_num, "| Validation Samples: ", len(dset) - train_num)

train_indicies = random.sample(dset_indicies, train_num)
validation_indicies = dset_indicies.difference(train_indicies)
training_data = [dset[i] for i in train_indicies]
validation_data = [dset[i] for i in validation_indicies]
print("Training and Validation datasets separated")

del dset_indicies
del train_indicies
del validation_indicies
del dset

import warnings
warnings.simplefilter('ignore')

import keras
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from generator_helper_functions import input_layer, naive_input_size, Generate_Naive_Feedforward_Contig

#opt = keras.optimizers.rmsprop()
opt = keras.optimizers.SGD()

batch_size = 128

model = Sequential()
model.add(Dense(90, activation='relu', input_shape=(naive_input_size,)))
model.add(Dense(90, activation='relu'))
model.add(Dense(1, activation='relu'))

model.compile(loss='poisson',
             optimizer=opt,
             metrics=['accuracy'])

print("Model Compiled")

train_gen = Generate_Naive_Feedforward_Contig(training_data, 
                                batch_size, 
                                (naive_input_size,), 
                                input_layer, 
                                lambda x: (x[2] - 1)/4)

validation_gen = Generate_Naive_Feedforward_Contig(validation_data, 
                                batch_size, 
                                (naive_input_size,), 
                                input_layer, 
                                lambda x: (x[2] - 1 )/4)

checkpointer = ModelCheckpoint(filepath='models/naive_feedforward_regression_poisson_90_90.hdf5', verbose=1, save_best_only=True)
model.fit_generator(train_gen,
                    epochs=1,
                    steps_per_epoch=500,
                    validation_data=validation_gen,
                    validation_steps=10,
                    callbacks=[checkpointer]
                    )